In [ ]:
# %pip install -q requests 'smolagents[litellm]' openai-agents openai python-dotenv

In [ ]:
import requests
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Hugging Face API configuration
HF_API_URL = "https://agents-course-unit4-scoring.hf.space"

In [ ]:
def get_random_question():
    """Fetch a random question from GAIA benchmark"""
    response = requests.get(f"{HF_API_URL}/random-question")
    return response.json() if response.status_code == 200 else None

def solve_gaia_task(question_data: dict) -> str:
    """Process GAIA question with strict answer formatting"""
    # Build precise prompt
    prompt = f"""Analyze and solve the following question. 
    Return ONLY the final answer in comma-separated alphabetical order, NO explanations.
    
    Question:
    {question_data['question']}"""
    
    # Get AI response
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # Use current production model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        max_tokens=50
    )
    
    # Extract and format answer
    raw_answer = response.choices[0].message.content
    return format_gaia_answer(raw_answer)

def format_gaia_answer(raw: str) -> str:
    """Enforce GAIA answer formatting rules"""
    # Remove non-alphanumeric except commas
    cleaned = "".join([c for c in raw if c.isalnum() or c in ", "])
    # Split, sort, and join elements
    elements = [e.strip().lower() for e in cleaned.split(",") if e.strip()]
    return ",".join(sorted(set(elements)))  # Remove duplicates and sort

# Execution flow
if __name__ == "__main__":
    question = get_random_question()
    if question:
        answer = solve_gaia_task(question)
        print(f"Task ID: {question['task_id']}")
        print(f"Formatted Answer: {answer}")  # Should output "b,e" for sample question
    else:
        print("Failed to fetch question")


In [ ]:
def submit_answers(answers: list):
    payload = {
        "username": "Shreyashgupta5",
        "agent_code": "https://huggingface.co/spaces/Shreyashgupta5/ai_agents_course",
        "answers": [{
            "task_id": q["task_id"],
            "submitted_answer": answer
        } for q, answer in answers]
    }
    response = requests.post(f"{HF_API_URL}/submit", json=payload)
    return response.json()
